### Introduction
Experiment with different search algorithms and heuristics for a agent that performs progression search to solve planning problems. Use the results to answer questions about designing planning systems

### Analyze the search complexity as a function of domain size, search algorithm, and heuristic.

- The chart or table includes data for all search & heuristic combinations for air cargo problems 1 and 2
- The chart or table includes data at least one uninformed search, two heuristics with greedy best first search, and two heuristics with A* on air cargo problems 3 and 4
- Report includes at least a one paragraph discussion of these results that analyzes the growth trends as the problem size increases

In [1]:
import run_search as rs
import _utils as utils
from timeit import default_timer as timer


def main(p_choices, s_choices):
    problems = [rs.PROBLEMS[i-1] for i in map(int, p_choices)]
    searches = [rs.SEARCHES[i-1] for i in map(int, s_choices)]

    result = None
    for pname, problem_fn in problems:
        for sname, search_fn, heuristic in searches:
            problem_instance = problem_fn()
            heuristic_fn = None if not heuristic else getattr(problem_instance, heuristic)
            result = run_search(problem_instance, search_fn, heuristic_fn)
    return result

def run_search(problem, search_function, parameter=None):
    ip = utils.PrintableProblem(problem)
    start = timer()
    if parameter is not None:
        node = search_function(ip, parameter)
    else:
        node = search_function(ip)
    end = timer()
    return (ip, len(node.solution()), (end - start))

problem, plan_length, elapsed_time  = main([1], [1])
print("########## Actions   Expansions   Goal Tests   New Nodes")
print(f"Problem: {problem}")
print(f"Plan length: {plan_length}")
print(f"Elapsed time: {elapsed_time}")


########## Actions   Expansions   Goal Tests   New Nodes
Problem:     20          43          56         178    
Plan length: 6
Elapsed time: 0.008439100000000366
